<a href="https://colab.research.google.com/github/fishenzone/portfolio/blob/main/projects/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instructions

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

##Load libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

## Import data

In [2]:
# !wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

# !unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

print(df_ratings.tail(),'\n')
df_books.tail()

           user         isbn  rating
1149775  276704   1563526298     9.0
1149776  276706   0679447156     0.0
1149777  276709   0515107662    10.0
1149778  276721   0590442449    10.0
1149779  276723  05162443314     8.0 



,isbn,title,author
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato
271378,0767409752,A Guided Tour of Rene Descartes' Meditations o...,Christopher Biffle


## Cleaning data

In [4]:
# Merge, and check the null
books_ratings_joined = pd.merge(df_ratings, df_books, on='isbn').drop(columns=['author'])

print(books_ratings_joined.isnull().sum(),'\n')
books_ratings_joined.tail()

user      0
isbn      0
rating    0
title     0
dtype: int64 



,user,isbn,rating,title
1031170,276688,0517145553,0.0,Mostly Harmless
1031171,276688,1575660792,7.0,Gray Matter
1031172,276690,0590907301,0.0,Triplet Trouble and the Class Trip (Triplet Tr...
1031173,276704,0679752714,0.0,A Desert of Pure Feeling (Vintage Contemporaries)
1031174,276704,0806917695,5.0,Perplexing Lateral Thinking Puzzles: Scholasti...


In [7]:
# Remove users with < 200 ratings and books with < 100 ratings simultaneously to ensure statistical significance
users_count = books_ratings_joined.user.value_counts()
mask_users = books_ratings_joined.user.isin(users_count[users_count >= 200].index)
books_count = books_ratings_joined.isbn.value_counts()
mask_books = books_ratings_joined.isbn.isin(books_count[books_count >= 100].index)

# Remove duplicates when the user has reviewed the same book more than one time 
books_ratings_joined_clean = books_ratings_joined[mask_users & mask_books]\
                             .drop_duplicates(subset=['user', 'title'], keep='first')\
                             .copy()

## Build model

In [8]:
df_knn = books_ratings_joined_clean.pivot(index='title', columns='user', values='rating').fillna(0)
ratings = csr_matrix(df_knn.values)

model = NearestNeighbors(metric='cosine')
model.fit(ratings)

NearestNeighbors(metric='cosine')

In [25]:
# function to return recommended books - this will be tested
def get_recommends(title = ''):
  try:
    book = df_knn.loc[title]
  except KeyError as e:
    print(f'The given book: {e} does not exist')

  dtc, idx = model.kneighbors([book.values], n_neighbors = 6)

  recommended_books = pd.DataFrame({
      'title'   : df_knn.index[idx[0]],
      'distance': dtc[0] 
  }).sort_values(by='distance', ascending=False).values

  return [title, recommended_books]

## Testing

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [26]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ['Tis: A Memoir', 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([['Tis: A Memoir', 0.7937096357345581],
       ['The Weight of Water', 0.7678344249725342],
       ['The Surgeon', 0.7669050693511963],
       ['I Know This Much Is True', 0.7646420001983643],
       ['The Lovely Bones: A Novel', 0.715856671333313],
       ["Where the Heart Is (Oprah's Book Club (Paperback))",
        1.7881393432617188e-07]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
